In [19]:
# importing libraries
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.ops import sigmoid
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2

In [6]:
# loading model 
model = load_model('./best_model_dc.keras')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# dimentions of the image
img_width, img_height = 150, 150
batch_size = 32

# preparing 
test_dir = './datasets/cats-dogs/test/'

test_datagen = ImageDataGenerator(
    1. / 255
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'binary'
)

Found 700 images belonging to 2 classes.


In [8]:

# evaluating the model
loss, accuracy = model.evaluate(test_generator)
print(f"Loss:{loss:.4f}, Accuracy:{accuracy*100:.2f}%")


/home/zikrulla/ML_DL_project/venv/lib/python3.12/site-packages/keras/src/legacy/preprocessing/image.py:1263: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/home/zikrulla/ML_DL_project/venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


22/22 ━━━━━━━━━━━━━━━━━━━━ 34s 1s/step - accuracy: 0.8315 - loss: 37.1671
Loss:36.4623, Accuracy:83.29%


In [22]:
# predict the image 

img = image.load_img('./catt.jpg', target_size=(img_width, img_height))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

images = np.vstack([x])
classes = model.predict(images)
#score = float(sigmoid(classes[0][0]))
#print(classes)

if classes == 1:
    print('It likes a dog!')
else: 
    print('It likes a cat!')



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
It likes a cat!


In [34]:
# drawing a bounbox on images
# loading an image 
image = Image.open('.//datasets/cats-dogs/test/dogs/27.jpg')

predictions = [
    # Sample predictions (x_min, y_min, width, height, confidence, class_id)
    [100, 150, 200, 250, 0.85, 1],
    #[300, 200, 150, 100, 0.90, 0],
]

# defining classes names
class_names = {
    0:"Cat", 
    1:"Dog"
}

# creating drowing context

draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

for pred in predictions:
    x_min, y_min, width, height, confidence, class_id = pred
    x_max, y_max = x_min + width, y_min + height

    draw.rectangle([x_min, y_min, x_max, y_max], outline='green', width=2)

    # adding labels
    label = f"{class_names[class_id]}: {confidence:.2f}"
    draw.text((x_min, y_min - 10), label, fill='green', font=font)

# save or show the image
image.save('predicted_image.jpg')

In [ ]:
#*************************************************************chatgpt

In [23]:

def preprocess_image(image_path, target_size=(224, 224)):
    image = cv2.imread(image_path)
    image = cv2.resize(image, target_size)
    image = image / 255.0  # Normalize pixel values between 0 and 1
    return image

def preprocess_bboxes(bboxes, original_size, target_size=(224, 224)):
    # Convert bounding boxes to the target size (scale bounding box)
    width_scale = target_size[0] / original_size[0]
    height_scale = target_size[1] / original_size[1]
    bboxes = np.array(bboxes) * [width_scale, height_scale, width_scale, height_scale]
    return bboxes


In [24]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout

def build_detection_model(input_shape=(224, 224, 3)):
    model = Sequential()

    # Convolutional layers
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))

    # Output 4 values (x, y, width, height) representing the bounding box
    model.add(Dense(4))

    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model


In [26]:

# Assume you have `images` as your input images and `bboxes` as their corresponding bounding boxes
datagen = ImageDataGenerator(rescale=1.0 / 255, validation_split=0.2)

train_gen = datagen.flow(images, bboxes, batch_size=32, subset='training')
val_gen = datagen.flow(images, bboxes, batch_size=32, subset='validation')

model = build_detection_model()

# Train the model
model.fit(train_gen, validation_data=val_gen, epochs=10)


NameError: name 'bboxes' is not defined

In [ ]:
def predict_bounding_box(image_path, model):
    image = preprocess_image(image_path)
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    pred_bbox = model.predict(image)[0]  # Model outputs bounding box coordinates
    return pred_bbox

image_path = 'test_image.jpg'
predicted_bbox = predict_bounding_box(image_path, model)
print("Predicted Bounding Box: ", predicted_bbox)


In [ ]:
def draw_bounding_box(image_path, bbox):
    image = cv2.imread(image_path)
    x, y, w, h = bbox

    start_point = (int(x), int(y))
    end_point = (int(x + w), int(y + h))

    color = (255, 0, 0)  # Blue color for the bounding box
    thickness = 2

    image = cv2.rectangle(image, start_point, end_point, color, thickness)
    cv2.imshow('Detected Image', image)
    cv2.waitKey(0)

image_path = 'test_image.jpg'
draw_bounding_box(image_path, predicted_bbox)


In [ ]:
val_loss, val_acc = model.evaluate(val_gen)
print(f"Validation Loss: {val_loss}, Validation Accuracy: {val_acc}")


In [ ]:
#*************************************************************chatgpt